# Estimation of AML Mutation Rates

Import necessary libraries:

In [13]:
import fitree
import pandas as pd
from pyfaidx import Fasta

Load the AML tree data:

In [14]:
AML_cohort = fitree.load_cohort_from_json("../data/AML_morita/AML_cohort_Morita_2020.json")

All 31 genes:

In [55]:
AML_genes = AML_cohort.mutation_labels
print(AML_genes)

['FLT3', 'NPM1', 'WT1', 'DNMT3A', 'KRAS', 'NRAS', 'RUNX1', 'IDH1', 'IDH2', 'PTPN11', 'SRSF2', 'ASXL1', 'BCOR', 'STAG2', 'TP53', 'U2AF1', 'SF3B1', 'TET2', 'CSF3R', 'JAK2', 'GATA2', 'EZH2', 'PPM1D', 'SETBP1', 'KIT', 'CBL', 'PHF6', 'MYC', 'ETV6', 'MPL', 'SMC3']


Load the variant information:

In [16]:
AML_variants = pd.read_csv("../data/AML_morita/AML_variants.csv")

Filter for those that have been validated:

In [17]:
AML_variants = AML_variants[AML_variants["validated"] == "yes"]

In [18]:
AML_variants

,sample ID,gene,chr,start,ref allele,alt allele,amino acid change,function,exonic function,mutated %,scDNA-seq VAF,WT\n(No. of cells),Het\n(No. of cells),Homo\n(No. of cells),Missing\n(No. of cells),validated,validation method,bulk VAF
0,AML-01-001,FLT3,13,28608259,X,XAACTCTAAATTTTCTCTTGGAAACTCCCATTTGAGATCATATTCA...,p.Y599_D600delinsX,exonic,stopgain,0.935200,0.7949,239,8444,2190,498,yes,bulk-NGS,0.1186
1,AML-01-001,NPM1,5,170837544,T,TCTGC,p.L287fs,exonic,frameshift insertion,0.424900,0.4608,1285,4226,606,5254,yes,bulk-NGS,0.1828
2,AML-01-001,WT1,11,32417913,C,CGTACAAGA,p.R380fs,exonic,frameshift insertion,0.003400,0.0015,10779,39,0,553,yes,bulk-NGS in paired sample,0
3,AML-01-001,WT1,11,32417943,C,CG,p.R370fs,exonic,frameshift insertion,0.883600,0.4865,568,9799,248,756,yes,bulk-NGS,0.236
4,AML-02-001,DNMT3A,2,25457242,C,T,p.R882H,exonic,nonsynonymous SNV,0.760300,0.4639,468,5861,169,1433,yes,bulk-NGS,0.4239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,AML-122-001,SF3B1,2,198267359,C,G,p.K666N,exonic,nonsynonymous SNV,0.903884,0.5040,197,3497,133,189,yes,clinical sequencing,not available
539,AML-123-001,DNMT3A,2,25457243,G,A,p.R882C,exonic,nonsynonymous SNV,0.631245,0.3954,643,1495,97,287,yes,clinical sequencing,not available
540,AML-123-001,RUNX1,21,36231774,G,A,p.R204X,exonic,stopgain,0.420698,0.2909,1061,1031,30,400,yes,clinical sequencing,not available
541,AML-123-001,TET2,4,106157248,C,CAA,p.H717fs,exonic,frameshift insertion,0.706186,0.3574,620,1751,30,121,yes,clinical sequencing,not available


Load haploid trinucleotide-context-site-specific mutation rates:

In [19]:
site_mutation_rates = pd.read_csv("../data/AML_morita/Watson_2020_trinucleotide_mutation_rates.csv")

In [35]:
site_mutation_rates

,Site,Mutation_rate
0,A[C>A]A,1.332230e-09
1,A[C>A]C,9.481170e-10
2,A[C>A]G,1.064910e-09
3,A[C>A]T,7.585960e-10
4,C[C>A]A,1.538010e-09
...,...,...
187,A[A>C]C,1.688930e-10
188,T[A>C]A,1.554360e-10
189,G[A>C]A,1.574550e-10
190,C[A>C]A,2.836760e-10


Load human genome file (hg19):

In [3]:
hg19_fasta = Fasta("../data/AML_morita/GRCh37.p13.genome.fa")

In [11]:
chrom = 'chr2'
start = 25457242 - 2
end = 25457242 + 2

# Extract the sequence
sequence = hg19_fasta[chrom][start:end]
print(sequence)

GCGG


For each variant, check whether it is an SNV. If yes, then query the trinucleotide context based on the columns "chr" and "start".

In [38]:
# For each variant, check whether it is an SNV. If yes, then query the trinucleotide context based on the columns "chr" and "start".
# The trinucleotide context is defined as the 3-mer sequence around the variant, where the variant is in the middle.
# Add a new column "tri_context" to the DataFrame AML_variants.
# Add also site-specific mutation rates to the DataFrame AML_variants.
# The site-specific mutation rates are stored in the DataFrame site_mutation_rates.
# The DataFrame site_mutation_rates has the following columns: "Site" and "Mutation_rate".
AML_variants["tri_context"] = ""
AML_variants["site_specific_mutation_rate"] = 0.0
for index, row in AML_variants.iterrows():
	if row["exonic function"] == "nonsynonymous SNV":
		chrom = "chr" + row["chr"]
		start = row["start"] - 2
		end = row["start"] + 1
		sequence = hg19_fasta[chrom][start:end]
		p0 = sequence[0]
		p1 = sequence[1]
		p2 = sequence[2]
		tri_context = f"{p0}[{p1}>{row['alt allele']}]{p2}"
		tri_rate = site_mutation_rates[site_mutation_rates["Site"] == tri_context]["Mutation_rate"].values[0]
		print(f"ref = {row['ref allele']}, alt = {row['alt allele']}, queried = {sequence}, tri_context = {tri_context}, tri_rate = {tri_rate}")
		AML_variants.at[index, "tri_context"] = tri_context
		AML_variants.at[index, "site_specific_mutation_rate"] = tri_rate
		

ref = C, alt = T, queried = GCG, tri_context = G[C>T]G, tri_rate = 1.88229e-08
ref = T, alt = A, queried = ATC, tri_context = A[T>A]C, tri_rate = 8.09824e-10
ref = G, alt = T, queried = TGT, tri_context = T[G>T]T, tri_rate = 1.33223e-09
ref = C, alt = G, queried = TCT, tri_context = T[C>G]T, tri_rate = 4.69475e-10
ref = A, alt = T, queried = TAT, tri_context = T[A>T]T, tri_rate = 4.25782e-10
ref = G, alt = A, queried = CGA, tri_context = C[G>A]A, tri_rate = 1.20044e-08
ref = C, alt = T, queried = ACC, tri_context = A[C>T]C, tri_rate = 3.17573e-09
ref = T, alt = A, queried = TTG, tri_context = T[T>A]G, tri_rate = 2.1756e-10
ref = C, alt = G, queried = TCT, tri_context = T[C>G]T, tri_rate = 4.69475e-10
ref = G, alt = T, queried = GGG, tri_context = G[G>T]G, tri_rate = 8.14186e-10
ref = G, alt = A, queried = CGT, tri_context = C[G>A]T, tri_rate = 3.26929e-08
ref = C, alt = T, queried = CCG, tri_context = C[C>T]G, tri_rate = 1.41512e-08
ref = T, alt = G, queried = ATC, tri_context = A[T>G]

In [40]:
# For all other variants, set the site-specific mutation rate to 2.7e-9.
AML_variants["site_specific_mutation_rate"] = AML_variants["site_specific_mutation_rate"].replace(0.0, 2.7e-9)

In [41]:
AML_variants

,sample ID,gene,chr,start,ref allele,alt allele,amino acid change,function,exonic function,mutated %,scDNA-seq VAF,WT\n(No. of cells),Het\n(No. of cells),Homo\n(No. of cells),Missing\n(No. of cells),validated,validation method,bulk VAF,tri_context,site_specific_mutation_rate
0,AML-01-001,FLT3,13,28608259,X,XAACTCTAAATTTTCTCTTGGAAACTCCCATTTGAGATCATATTCA...,p.Y599_D600delinsX,exonic,stopgain,0.935200,0.7949,239,8444,2190,498,yes,bulk-NGS,0.1186,,2.700000e-09
1,AML-01-001,NPM1,5,170837544,T,TCTGC,p.L287fs,exonic,frameshift insertion,0.424900,0.4608,1285,4226,606,5254,yes,bulk-NGS,0.1828,,2.700000e-09
2,AML-01-001,WT1,11,32417913,C,CGTACAAGA,p.R380fs,exonic,frameshift insertion,0.003400,0.0015,10779,39,0,553,yes,bulk-NGS in paired sample,0,,2.700000e-09
3,AML-01-001,WT1,11,32417943,C,CG,p.R370fs,exonic,frameshift insertion,0.883600,0.4865,568,9799,248,756,yes,bulk-NGS,0.236,,2.700000e-09
4,AML-02-001,DNMT3A,2,25457242,C,T,p.R882H,exonic,nonsynonymous SNV,0.760300,0.4639,468,5861,169,1433,yes,bulk-NGS,0.4239,G[C>T]G,1.882290e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,AML-122-001,SF3B1,2,198267359,C,G,p.K666N,exonic,nonsynonymous SNV,0.903884,0.5040,197,3497,133,189,yes,clinical sequencing,not available,T[C>G]T,4.694750e-10
539,AML-123-001,DNMT3A,2,25457243,G,A,p.R882C,exonic,nonsynonymous SNV,0.631245,0.3954,643,1495,97,287,yes,clinical sequencing,not available,C[G>A]G,1.415120e-08
540,AML-123-001,RUNX1,21,36231774,G,A,p.R204X,exonic,stopgain,0.420698,0.2909,1061,1031,30,400,yes,clinical sequencing,not available,,2.700000e-09
541,AML-123-001,TET2,4,106157248,C,CAA,p.H717fs,exonic,frameshift insertion,0.706186,0.3574,620,1751,30,121,yes,clinical sequencing,not available,,2.700000e-09


In [46]:
# Get all unique combinations of (gene, amino acid change, site_specific_mutation_rate)
AML_variants_unique = AML_variants.drop_duplicates(subset=["gene", "amino acid change", "site_specific_mutation_rate"])
AML_variants_unique = AML_variants_unique[["gene", "amino acid change", "exonic function", "site_specific_mutation_rate"]]

In [54]:
AML_variants_unique.query("gene == 'NRAS'")

,gene,amino acid change,exonic function,site_specific_mutation_rate
14,NRAS,p.G12D,nonsynonymous SNV,3.175730e-09
25,NRAS,p.G13D,nonsynonymous SNV,3.175730e-09
27,NRAS,p.Q61H,nonsynonymous SNV,2.521550e-10
29,NRAS,p.G13R,nonsynonymous SNV,4.643030e-10
81,NRAS,p.G13V,nonsynonymous SNV,9.481170e-10
88,NRAS,p.G13C,nonsynonymous SNV,1.538010e-09
89,NRAS,p.G12V,nonsynonymous SNV,9.481170e-10
91,NRAS,p.G12C,nonsynonymous SNV,9.825080e-10
92,NRAS,p.G12S,nonsynonymous SNV,4.379090e-09
162,NRAS,p.G12A,nonsynonymous SNV,4.832630e-10


In [56]:
# For each gene, sum up the site-specific mutation rates.
AML_variants_sum = AML_variants.groupby("gene").agg({"site_specific_mutation_rate": "sum"}).reset_index()

In [57]:
AML_variants_sum

,gene,site_specific_mutation_rate
0,ASXL1,4.590000e-08
1,BCOR,1.350000e-08
2,CBL,1.630361e-08
3,CSF3R,2.700000e-09
4,DNMT3A,6.364295e-07
5,ETV6,2.700000e-09
6,EZH2,5.053607e-08
7,FLT3,1.423273e-07
8,GATA2,6.238082e-09
9,IDH1,3.875233e-07


In [65]:
# Reset AML_cohort.mu_vec based on the calculated mutation rates
for idx, gene in enumerate(AML_genes):
	new_mu = AML_variants_sum[AML_variants_sum["gene"] == gene]["site_specific_mutation_rate"].values[0]
	print(f"gene = {gene}, old mu = {AML_cohort.mu_vec[idx]}, new mu = {new_mu}")
	AML_cohort.mu_vec[idx] = new_mu

gene = FLT3, old mu = 1.4232733200000001e-07, new mu = 1.4232733200000001e-07
gene = NPM1, old mu = 1.3230000000000001e-07, new mu = 1.3230000000000001e-07
gene = WT1, old mu = 1.049503e-07, new mu = 1.049503e-07
gene = DNMT3A, old mu = 6.36429535e-07, new mu = 6.36429535e-07
gene = KRAS, old mu = 7.4349048e-08, new mu = 7.4349048e-08
gene = NRAS, old mu = 1.55749763e-07, new mu = 1.55749763e-07
gene = RUNX1, old mu = 1.1756328e-07, new mu = 1.1756328e-07
gene = IDH1, old mu = 3.8752333000000003e-07, new mu = 3.8752333000000003e-07
gene = IDH2, old mu = 4.2790116e-07, new mu = 4.2790116e-07
gene = PTPN11, old mu = 3.4190748000000004e-08, new mu = 3.4190748000000004e-08
gene = SRSF2, old mu = 2.88385e-08, new mu = 2.88385e-08
gene = ASXL1, old mu = 4.59e-08, new mu = 4.59e-08
gene = BCOR, old mu = 1.3500000000000002e-08, new mu = 1.3500000000000002e-08
gene = STAG2, old mu = 1.08e-08, new mu = 1.08e-08
gene = TP53, old mu = 5.1095175e-08, new mu = 5.1095175e-08
gene = U2AF1, old mu = 9.

In [66]:
fitree.save_cohort_to_json(AML_cohort, "../data/AML_morita/AML_cohort_Morita_2020.json")